In [1]:
import os

In [2]:
import os
import random
import shutil

root_dir = "raw_dataset/rgbd-scenes-v2/imgs"
out_images = "raw_dataset/filter_filterrgbd-scenes-v2/images"
out_labels = "raw_dataset/filter_filterrgbd-scenes-v2/labels"

os.makedirs(out_images, exist_ok=True)
os.makedirs(out_labels, exist_ok=True)

scenes = sorted([d for d in os.listdir(root_dir) if d.startswith("scene_")])

for scene in scenes:
    scene_path = os.path.join(root_dir, scene)
    files = sorted([f for f in os.listdir(scene_path) if f.endswith("-color.png")])
    
    # Lấy id chung cho color/depth
    ids = [f.split("-")[0] for f in files]
    random.shuffle(ids)
    ids = ids[:200]

    for idx in ids:
        color_file = f"{idx}-color.png"
        depth_file = f"{idx}-depth.png"

        # Tạo tên mới
        new_color = f"{scene}_{color_file}"
        new_depth = f"{scene}_{depth_file}"

        # Copy vào images (color) và labels (depth)
        shutil.copy(os.path.join(scene_path, color_file),
                    os.path.join(out_images, new_color))
        shutil.copy(os.path.join(scene_path, depth_file),
                    os.path.join(out_labels, new_depth))


In [3]:
import os
import random
import shutil

src_images = "raw_dataset/filter_filterrgbd-scenes-v2/images"
src_labels = "raw_dataset/filter_filterrgbd-scenes-v2/labels"
dst_root = "dataset_ours/rgbd-scenes-v2"

splits = {"train": 0.8, "val": 0.1, "test": 0.1}

# Tạo thư mục đích
for split in splits:
    os.makedirs(os.path.join(dst_root, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(dst_root, split, "labels"), exist_ok=True)

# Lấy danh sách file ảnh (color)
files = sorted(os.listdir(src_images))
random.shuffle(files)

n_total = len(files)
n_train = int(n_total * splits["train"])
n_val = int(n_total * splits["val"])

split_files = {
    "train": files[:n_train],
    "val": files[n_train:n_train + n_val],
    "test": files[n_train + n_val:]
}

for split, file_list in split_files.items():
    for fname in file_list:
        # Tìm depth file tương ứng
        depth_name = fname.replace("-color.png", "-depth.png")

        # Copy
        shutil.copy(os.path.join(src_images, fname),
                    os.path.join(dst_root, split, "images", fname))
        shutil.copy(os.path.join(src_labels, depth_name),
                    os.path.join(dst_root, split, "labels", depth_name))
